In [19]:
import pandas as pd
import numpy as np
import joblib

In [20]:
# Загрузка модели
model = joblib.load("../models/model_oned.pkl")

# Загрузка тестовых данных
test_data = pd.read_excel("../data/test.xlsx")
test_data['dt'] = pd.to_datetime(test_data['dt'])

In [21]:
# Создание лагов
lags = 6
for i in range(1, lags + 1):
    test_data[f'lag_{i}'] = test_data['Цена на арматуру'].shift(i)

test_data.fillna(method='bfill', inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_10048\1832702259.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_data.fillna(method='bfill', inplace=True)


In [22]:
# Функция прогнозирования на 6 недель вперед
def predict_n_weeks_ahead(model, features, n_weeks=6):
    predictions = []
    
    # Текущие значения лагов
    current_features = features.copy()

    for _ in range(n_weeks):
        # Делаем предсказание, передавая в модель все лаги
        prediction = model.predict([current_features])[0]
        predictions.append(prediction)

        # Обновляем лаги: удаляем самый старый, добавляем новое предсказание
        current_features = np.roll(current_features, shift=-1)
        current_features[-1] = prediction
    
    return predictions

In [23]:
def determine_n_weeks(predictions):
    n_weeks = 1
    for i in range(1, len(predictions)):
        if predictions[i] < predictions[i - 1]:  # Если цена падает
            n_weeks += 1
        elif predictions[i] == predictions[i - 1]:  # Если цена не меняется
            continue  # Не увеличиваем n_weeks
        else:  # Если цена растет
            break  # Прерываем цикл, так как тренд уже сменился на рост
    return min(n_weeks, 6)

In [24]:
# Добавляем колонку с рекомендацией по объему тендера
test_data['N_weeks'] = test_data.apply(
    lambda row: determine_n_weeks(predict_n_weeks_ahead(model, row.iloc[2:].values)), axis=1
)

In [25]:
# Переименовываем колонку N_weeks в "Объем"
test_data.rename(columns={'N_weeks': 'Объем'}, inplace=True)

# Сохраняем только нужные колонки
columns_to_keep = ['dt', 'Цена на арматуру', 'Объем']  # Сохраняем только нужные колонки
test_data = test_data[columns_to_keep]

In [26]:
# Сохраняем результат в новый файл
test_data.to_excel("../data/test_with_recommendations.xlsx", index=False)